In [29]:
import pandas as pd
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV, SelectKBest, f_classif

from sklearn.tree import DecisionTreeClassifier

from hyperopt import hp, fmin, tpe
from numpy import random

In [2]:
data = pd.read_csv('heart.csv')
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
age         303 non-null int64
sex         303 non-null int64
cp          303 non-null int64
trestbps    303 non-null int64
chol        303 non-null int64
fbs         303 non-null int64
restecg     303 non-null int64
thalach     303 non-null int64
exang       303 non-null int64
oldpeak     303 non-null float64
slope       303 non-null int64
ca          303 non-null int64
thal        303 non-null int64
target      303 non-null int64
dtypes: float64(1), int64(13)
memory usage: 33.2 KB


In [4]:
x = data.iloc[:,:13]
y = data.iloc[:, 13:]

In [5]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.2, random_state = 100)
xtest.head()
#xtrain.columns

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
69,62,0,0,124,209,0,1,163,0,0.0,2,0,2
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3
220,63,0,0,150,407,0,0,154,0,4.0,1,3,3
134,41,0,1,126,306,0,1,163,0,0.0,2,0,2
7,44,1,1,120,263,0,1,173,0,0.0,2,0,3


In [6]:
scaler = StandardScaler()
xtrain[['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal']] = scaler.fit_transform(xtrain[['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal']])

xtest[['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal']] = scaler.fit_transform(xtest[['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal']])

C:\Users\kalya\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\kalya\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\kalya\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [7]:
xtrain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 242 entries, 184 to 8
Data columns (total 13 columns):
age         242 non-null float64
sex         242 non-null float64
cp          242 non-null float64
trestbps    242 non-null float64
chol        242 non-null float64
fbs         242 non-null float64
restecg     242 non-null float64
thalach     242 non-null float64
exang       242 non-null float64
oldpeak     242 non-null float64
slope       242 non-null float64
ca          242 non-null float64
thal        242 non-null float64
dtypes: float64(13)
memory usage: 26.5 KB


In [8]:
skb = SelectKBest(k=4)
skb.fit(xtrain, ytrain)
skb.get_support()

C:\Users\kalya\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([False, False,  True, False, False, False, False, False,  True,
        True, False, False,  True])

In [14]:
estimator = DecisionTreeClassifier()
rfecv = RFECV(estimator)
rfecv.fit(xtrain, ytrain)
col = rfecv.get_support()

C:\Users\kalya\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\kalya\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [18]:
features = xtrain.columns
new_features = []
for bool, feature in zip(col, features):
    if bool:
        new_features.append(feature)

new_features

['age', 'sex', 'cp', 'trestbps', 'chol', 'thalach', 'oldpeak', 'ca', 'thal']

In [19]:
train2 = pd.DataFrame(xtrain, columns=new_features)
train2.head()

,age,sex,cp,trestbps,chol,thalach,oldpeak,ca,thal
184,-0.497803,0.689645,-0.964689,1.044239,-0.083717,-0.979764,1.391297,-0.718858,1.158151
19,1.647367,-1.450022,1.870794,0.462638,-0.165905,0.048785,0.693305,1.292282,-0.550828
118,-0.949417,-1.450022,-0.019528,-1.572968,-0.885052,0.987895,-0.877177,-0.718858,-0.550828
41,-0.723610,0.689645,-0.019528,-0.118964,-0.042622,1.345651,-0.702679,-0.718858,-0.550828
59,0.292523,-1.450022,-0.964689,-0.235284,1.149107,0.406541,-0.877177,0.286712,-0.550828


In [33]:
# WIth Bayesian Optimization and KNN
def objective_func(args):
    n_neighbors = args['n_neighbors']
    metric = args['metric']
    clf = KNeighborsClassifier(n_neighbors=n_neighbors, metric=metric, n_jobs = -1)
 
    #clf.fit(x_pca,y_train.values.ravel())    
    #y_pred = clf.predict(x_valpca)
    #f1 = -(f1_score(y_pred,y_val,  average='macro'))
    f1 = -(cross_val_score(clf, xtrain,ytrain.values.ravel(), cv=5, scoring = 'f1_macro').mean())
    return f1
space = {'n_neighbors': hp.choice('n_neighbors',np.arange(1,5, step=1)),
        'metric':hp.choice('metric', ["euclidean","manhattan"])}

best_classifier = fmin(objective_func, space, algo=tpe.suggest, max_evals=100)
print(best_classifier)

100%|██████████| 100/100 [00:15<00:00,  6.19it/s, best loss: -0.8067600279607436]
{'metric': 0, 'n_neighbors': 3}
